In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from scienceworld import ScienceWorldEnv

from sources.scienceworld import load_step_function,parse_observation
from sources.agent import BDIAgent
from sources.bdi_components.inference import NLIModel
from sources.bdi_components.belief import State

import numpy as np
from os import listdir
from os.path import isfile, join
import re

results_df = pd.read_csv("../results/results_find-non-living-thing.csv")
# TODO ajustar dps o script
results_df.loc[results_df["plans_pct"] == 1, "plans_pct"] = 100
results_df.loc[results_df["plans_pct"] == 2, "plans_pct"] = 25
results_df.loc[results_df["plans_pct"] == 5, "plans_pct"] = 50
results_df.loc[results_df["plans_pct"] == 7, "plans_pct"] = 75

results_df.columns

Index(['num_bdi_actions', 'num_rl_actions', 'plan_found', 'variation', 'error',
       'bdi_score', 'rl_score', 'final_score', 'complete', 'num_plans',
       'plan_library_size', 'plans_pct', 'eps', 'drrn_model_file'],
      dtype='object')

- Heatmap comparing plans_pct vs eps_trained
- Num variations completed by using RL x BDI

In [2]:
results_df[['variation', 'plan_library_size', 'plans_pct']].drop_duplicates()

,variation,plan_library_size,plans_pct
0,225,12,0
1,226,13,0
2,227,14,0
3,228,15,0
4,229,16,0
...,...,...,...
3070,295,41,100
3071,296,41,100
3072,297,41,100
3073,298,41,100


In [3]:
projected_cols = ['plans_pct', 'eps']
aggregations = {'final_score': 'mean', 'rl_score': 'mean', 'bdi_score': 'mean', 'num_bdi_actions': 'mean', 'num_rl_actions': 'mean'}
results_df.groupby(projected_cols).agg(aggregations).reset_index()

,plans_pct,eps,final_score,rl_score,bdi_score,num_bdi_actions,num_rl_actions
0,0,58,19.186667,19.186667,0.000000,0.000000,50.000000
1,0,119,19.186667,19.186667,0.000000,0.000000,50.000000
2,0,181,23.400000,23.400000,0.000000,0.000000,50.000000
3,0,242,65.853333,65.853333,0.000000,0.000000,50.000000
4,0,302,67.746667,67.746667,0.000000,0.000000,50.000000
5,0,362,65.066667,65.066667,0.000000,0.000000,50.000000
6,0,421,65.853333,65.853333,0.000000,0.000000,50.000000
7,0,480,65.853333,65.853333,0.000000,0.000000,50.000000
8,0,535,65.853333,65.853333,0.000000,0.000000,50.000000
9,0,593,16.920000,16.920000,0.000000,0.000000,50.000000


In [9]:
plots_df = results_df[['final_score', 'eps', 'plans_pct']]
plots_df = plots_df.groupby(['eps', 'plans_pct']).agg(mean_score = ('final_score', 'mean'), max_score=('final_score', 'max'), min_score=('final_score', 'min'), stddev_score=('final_score', 'std')).reset_index()
plots_df['e_plus'] = plots_df['mean_score'] + plots_df['stddev_score']
plots_df['e_minus'] = plots_df['mean_score'] - plots_df['stddev_score']
plots_df

,eps,plans_pct,mean_score,max_score,min_score,stddev_score,e_plus,e_minus
0,58,0,19.186667,25,8,4.599334,23.786001,14.587333
1,58,25,42.906667,100,8,36.886302,79.792969,6.020365
2,58,50,65.533333,100,8,39.992342,105.525675,25.540992
3,58,75,83.146667,100,17,32.231717,115.378384,50.914949
4,58,100,98.640000,100,83,4.643042,103.283042,93.996958
5,119,0,19.186667,25,8,4.599334,23.786001,14.587333
6,119,25,42.906667,100,8,36.886302,79.792969,6.020365
7,119,50,65.533333,100,8,39.992342,105.525675,25.540992
8,119,75,83.146667,100,17,32.231717,115.378384,50.914949
9,119,100,98.640000,100,83,4.643042,103.283042,93.996958


In [29]:
all_scaters = []
for group, group_df in plots_df.groupby("plans_pct"):
    all_scaters.append(go.Scatter(name=f"Plans pct {group}",
        x=group_df['eps'],
        y=group_df['mean_score'],
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=group_df['stddev_score'],
            visible=True,
            thickness=0.5)
    ))

fig = go.Figure(data=all_scaters)
fig.update_layout(width=1000,height=1000)
fig.show()

In [24]:
curve_plot_df = results_df[['plans_pct', 'bdi_score', 'rl_score']]
curve_plot_df = pd.melt(curve_plot_df, id_vars=['plans_pct'], value_vars=['bdi_score', 'rl_score'], var_name='agent', value_name='score')
#curve_plot_df['plans_pct'] = curve_plot_df['plans_pct'].astype(str)
curve_plot_df = curve_plot_df.groupby(['plans_pct', 'agent']).agg({'score':'mean'}).reset_index(drop=False)

fig = px.bar(curve_plot_df, x="plans_pct", y="score", color="agent", title="Score Distribution")
fig.update_layout(width=1000,height=1500)
fig.show()

In [15]:
curve_plot_df = results_df[['plan_library_size', 'eps', 'bdi_score', 'rl_score']]
curve_plot_df = pd.melt(curve_plot_df, id_vars=['plan_library_size', 'eps'], value_vars=['bdi_score', 'rl_score'], var_name='agent', value_name='score')
#curve_plot_df['plans_pct'] = curve_plot_df['plans_pct'].astype(str)
curve_plot_df = curve_plot_df.groupby(['plan_library_size', 'eps']).agg({'score':'mean'}).reset_index(drop=False)

fig = px.bar(curve_plot_df, x="plans_pct", y="score", color="agent", title="Score Distribution")
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['plan_library_size', 'eps', 'score'] but received: plans_pct

In [13]:
curve_plot_df = results_df[['plans_pct', 'bdi_score', 'rl_score']]
curve_plot_df = pd.melt(curve_plot_df, id_vars=['plans_pct'], value_vars=['bdi_score', 'rl_score'], var_name='agent', value_name='score')
#curve_plot_df['plans_pct'] = curve_plot_df['plans_pct'].astype(str)
curve_plot_df = curve_plot_df.groupby(['plans_pct', 'agent']).agg({'score':'mean'}).reset_index(drop=False)

fig = px.bar(curve_plot_df, x="plans_pct", y="score", color="agent", title="Score Distribution")
fig.show()

In [21]:
heatmap_df = results_df[['plan_library_size', 'eps', 'final_score']]
heatmap_df = heatmap_df.groupby(['plan_library_size', 'eps']).agg({'final_score': 'mean'}).reset_index()
heatmap_df = heatmap_df.pivot(index='plan_library_size', columns='eps')['final_score'].fillna(0)

fig = px.imshow(heatmap_df, x=heatmap_df.columns, y=heatmap_df.index)
fig.update_layout(width=1000,height=700)
fig.update_xaxes(type='category')
#fig.update_yaxes(type='category')
fig.show()

In [31]:
curve_points_df = results_df[['plans_pct', 'eps', 'final_score']]
curve_points_df = curve_points_df.groupby(['plans_pct', 'eps']).agg({'final_score': 'mean'}).reset_index()
fig = px.scatter(curve_points_df, x="plans_pct", y="eps", size='final_score')
fig.update_layout(width=700,height=700)
fig.show()